# Realizando os imports das libs necessárias para o download dos dados

In [20]:
# Adicionando os imports
import requests, zipfile, io
import pandas as pd
from datetime import datetime

import os
from google.cloud import bigquery

In [21]:
# Definindo a url para baixar os dados
zip_file_url = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/2024.zip'
data = requests.get(zip_file_url)

In [22]:
# Salvando os dados baixados do arquivo zip
z = zipfile.ZipFile(io.BytesIO(data.content))

In [23]:
# Descompactando o arquivo zip no diretório data
z.extractall('data')

In [24]:
# Acessando os dados da cidade de Goiânia
for arquivo in os.listdir('data/'):
    if arquivo.endswith('.CSV') and 'GOIANIA' in arquivo:
        print(arquivo)


INMET_CO_GO_A002_GOIANIA_01-01-2024_A_30-06-2024.CSV


In [25]:
# Lendo e carregando o dataset do csv
df = pd.read_csv('data/INMET_CO_GO_A002_GOIANIA_01-01-2024_A_30-06-2024.CSV', delimiter=';', encoding='unicode_escape', skiprows=8)
df.head()

,Data,Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (Kj/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19
0,2024/01/01,0000 UTC,0,"930,6","930,6","929,9",NaN,"23,6","19,5","24,4","23,6","20,2","19,5",79,77,78,297,"2,5",",3",NaN
1,2024/01/01,0100 UTC,0,"931,7","931,7","930,6",NaN,"23,2","19,8","23,6","23,1","19,8","19,3",81,77,81,357,"3,3",",2",NaN
2,2024/01/01,0200 UTC,0,"932,4","932,4","931,7",NaN,23,20,"23,2","22,9",20,"19,8",83,81,83,346,2,",4",NaN
3,2024/01/01,0300 UTC,0,932,"932,5",932,NaN,"22,3","19,9",23,"22,3",20,"19,8",86,83,86,347,2,",1",NaN
4,2024/01/01,0400 UTC,0,"931,3",932,"931,3",NaN,22,"19,5","22,4",22,"19,9","19,5",86,85,86,281,"1,3",0,NaN


# Tratamento dos dados:
### Removendo colunas inválidas
### Removendo dados faltantes
### Formatando e alterando tipos dos dados

In [26]:
# Deletando coluna sem dados
df.drop(['Unnamed: 19'], inplace = True, axis=1)

In [27]:
# Renomeando os nomes das colunas para melhor tratamento
df = df.rename(columns={
'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'PRECIPITACAO_TOTAL',
'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'PRESSAO_ATMOSFERICA_NIVEL_ESTACAO',
'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'PRESSAO_ATMOSFERICA_MAX_HORA_ANT',
'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'PRESSAO_ATMOSFERICA_MIN_HORA_ANT',
'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO',
'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'TEMPERATURA_AR',
'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'TEMPERATURA_PONTO_ORVALHO',
'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'TEMPERATURA_MAXIMA_HORA_ANT',
'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'TEMPERATURA_MINIMA_HORA_ANT',
'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'TEMPERATURA_ORVALHO_MAX_HORA_ANT',
'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'TEMPERATURA_ORVALHO_MIN_HORA_ANT',
'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'UMIDADE_REL_MAX_HORA_ANT',
'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'UMIDADE_REL_MIN_HORA_ANT',
'UMIDADE RELATIVA DO AR, HORARIA (%)': 'UMIDADE_RELATIVA_AR_HORARIA',
'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'VENTO_DIRECAO_HORARIA',
'VENTO, RAJADA MAXIMA (m/s)': 'VENTO_RAJADA_MAXIMA',
'VENTO, VELOCIDADE HORARIA (m/s)': 'VENTO_VELOCIDADE_HORARIA'
}, errors="raise")

In [28]:
# Juntando os dados de Data e Hora em somente uma coluna. Em seguida a coluna DATA_HORA é transformada em int para timestamp do banco.

df["DATA_HORA"] =  pd.to_datetime(df["Data"] + ' ' + df["Hora UTC"].str.strip(' UTC'), format="%Y/%m/%d %H%M")
df["DATA_HORA"] = df['DATA_HORA'].astype('int64') // 10**9
df.drop(['Data'], inplace = True, axis=1)
df.drop(['Hora UTC'], inplace = True, axis=1)

# Todos os dados que são do tipo object ou string, terão a troca de "," por "."
for column in df.columns:
      if df[column].dtype == 'object' :
        df[column] = df[column].str.replace(',', '.')
      df[column] = df[column].astype(float)

df.dtypes

PRECIPITACAO_TOTAL                   float64
PRESSAO_ATMOSFERICA_NIVEL_ESTACAO    float64
PRESSAO_ATMOSFERICA_MAX_HORA_ANT     float64
PRESSAO_ATMOSFERICA_MIN_HORA_ANT     float64
RADIACAO                             float64
TEMPERATURA_AR                       float64
TEMPERATURA_PONTO_ORVALHO            float64
TEMPERATURA_MAXIMA_HORA_ANT          float64
TEMPERATURA_MINIMA_HORA_ANT          float64
TEMPERATURA_ORVALHO_MAX_HORA_ANT     float64
TEMPERATURA_ORVALHO_MIN_HORA_ANT     float64
UMIDADE_REL_MAX_HORA_ANT             float64
UMIDADE_REL_MIN_HORA_ANT             float64
UMIDADE_RELATIVA_AR_HORARIA          float64
VENTO_DIRECAO_HORARIA                float64
VENTO_RAJADA_MAXIMA                  float64
VENTO_VELOCIDADE_HORARIA             float64
DATA_HORA                            float64
dtype: object

In [29]:
# Visualizando os nomes das colunas para conferência
df.columns

Index(['PRECIPITACAO_TOTAL', 'PRESSAO_ATMOSFERICA_NIVEL_ESTACAO',
       'PRESSAO_ATMOSFERICA_MAX_HORA_ANT', 'PRESSAO_ATMOSFERICA_MIN_HORA_ANT',
       'RADIACAO', 'TEMPERATURA_AR', 'TEMPERATURA_PONTO_ORVALHO',
       'TEMPERATURA_MAXIMA_HORA_ANT', 'TEMPERATURA_MINIMA_HORA_ANT',
       'TEMPERATURA_ORVALHO_MAX_HORA_ANT', 'TEMPERATURA_ORVALHO_MIN_HORA_ANT',
       'UMIDADE_REL_MAX_HORA_ANT', 'UMIDADE_REL_MIN_HORA_ANT',
       'UMIDADE_RELATIVA_AR_HORARIA', 'VENTO_DIRECAO_HORARIA',
       'VENTO_RAJADA_MAXIMA', 'VENTO_VELOCIDADE_HORARIA', 'DATA_HORA'],
      dtype='object')

In [30]:
# Deletando os dados faltantes
df = df.dropna()

# Salvando os dados no BigQuery

In [31]:
# Criando um client para acessar o BigQuery
client = bigquery.Client()

In [32]:
# Salvando no BigQuery os dados da tabela PRECIPITACAO
errors = client.insert_rows_json('mvp-data-engineer-v1.mvp3_dataset.PRECIPITACAO', df[['DATA_HORA','PRECIPITACAO_TOTAL','RADIACAO']].to_dict(orient='records'))
if errors == []:
    print("Dados PRECIPITACAO inseridos com sucesso.")
else:
    print("Erros ao inserir dados PRECIPITACAO:", errors)

Dados PRECIPITACAO inseridos com sucesso.


In [33]:
# Salvando no BigQuery os dados da tabela PRESSAO
errors = client.insert_rows_json('mvp-data-engineer-v1.mvp3_dataset.PRESSAO', df[['DATA_HORA','PRESSAO_ATMOSFERICA_NIVEL_ESTACAO','PRESSAO_ATMOSFERICA_MAX_HORA_ANT','PRESSAO_ATMOSFERICA_MIN_HORA_ANT']].to_dict(orient='records'))
if errors == []:
    print("Dados PRESSAO inseridos com sucesso.")
else:
    print("Erros ao inserir dados PRESSAO:", errors)

Dados PRESSAO inseridos com sucesso.


In [34]:
# Salvando no BigQuery os dados da tabela TEMPERATURA
errors = client.insert_rows_json('mvp-data-engineer-v1.mvp3_dataset.TEMPERATURA', df[['DATA_HORA','TEMPERATURA_AR','TEMPERATURA_PONTO_ORVALHO','TEMPERATURA_MAXIMA_HORA_ANT','TEMPERATURA_MINIMA_HORA_ANT','TEMPERATURA_ORVALHO_MAX_HORA_ANT','TEMPERATURA_ORVALHO_MIN_HORA_ANT']].to_dict(orient='records'))
if errors == []:
    print("Dados TEMPERATURA inseridos com sucesso.")
else:
    print("Erros ao inserir dados TEMPERATURA:", errors)

Dados TEMPERATURA inseridos com sucesso.


In [35]:
# Salvando no BigQuery os dados da tabela UMIDADE
errors = client.insert_rows_json('mvp-data-engineer-v1.mvp3_dataset.UMIDADE', df[['DATA_HORA','UMIDADE_REL_MAX_HORA_ANT','UMIDADE_REL_MIN_HORA_ANT','UMIDADE_RELATIVA_AR_HORARIA']].to_dict(orient='records'))
if errors == []:
    print("Dados UMIDADE inseridos com sucesso.")
else:
    print("Erros ao inserir dados UMIDADE:", errors)

Dados UMIDADE inseridos com sucesso.


In [36]:
# Salvando no BigQuery os dados da tabela VENTO
errors = client.insert_rows_json('mvp-data-engineer-v1.mvp3_dataset.VENTO', df[['DATA_HORA','VENTO_DIRECAO_HORARIA','VENTO_RAJADA_MAXIMA','VENTO_VELOCIDADE_HORARIA']].to_dict(orient='records'))
if errors == []:
    print("Dados VENTO inseridos com sucesso.")
else:
    print("Erros ao inserir dados VENTO:", errors)

Dados VENTO inseridos com sucesso.


# Executando uma consulta simples na tabela PRECIPITACAO para conferir que os dados foram salvos no BigQuery

In [37]:
%%bigquery
SELECT * FROM `mvp-data-engineer-v1.mvp3_dataset.PRECIPITACAO` LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

,DATA_HORA,PRECIPITACAO_TOTAL,RADIACAO
0,2024-01-01 09:00:00+00:00,0.0,7.9
1,2024-01-01 10:00:00+00:00,0.0,298.6
2,2024-01-01 11:00:00+00:00,0.0,1260.7
3,2024-01-01 12:00:00+00:00,0.0,1657.8
4,2024-01-01 13:00:00+00:00,0.0,2228.9
...,...,...,...
995,2024-03-15 19:00:00+00:00,9.2,911.9
996,2024-03-15 20:00:00+00:00,3.4,283.9
997,2024-03-15 21:00:00+00:00,0.0,190.8
998,2024-03-15 22:00:00+00:00,0.0,7.0


# Por fim, salvando os dados do Dataframe pandas no diretório de arquivos processados.

In [38]:
data_atual = datetime.today()
nome_diretorio = '{}-{}-{}-{}h-{}m-{}s'.format(data_atual.year, data_atual.month, data_atual.day, data_atual.hour, data_atual.minute, data_atual.second)

df.to_csv(f'gs://mvp-data-engineer-v1/processed/{nome_diretorio}/INMET_CO_GO_A002_GOIANIA.CSV', index=False, sep=';', encoding='utf-8')